In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from collections import Counter
from nltk.tokenize import word_tokenize
import torch

g:\Melody-Lyric-Generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [3]:
model_path = "./gpt2_model_trained_1"
model_1 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_1 = AutoTokenizer.from_pretrained(model_path)

In [4]:
model_path = "./gpt2_model_trained_2"
model_2 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_2 = AutoTokenizer.from_pretrained(model_path)

In [5]:
model_path = "./gpt2_model_trained_3"
model_3 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_3 = AutoTokenizer.from_pretrained(model_path)

In [6]:
model_path = "./gpt2_model_trained_4"
model_4 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_4 = AutoTokenizer.from_pretrained(model_path)

## Prompt

In [7]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_1 = tokenizer_1(prompt, return_tensors="pt").input_ids

output = model_1.generate(
    input_ids=input_ids_1,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_1.eos_token_id,
)

generated_text_1 = tokenizer_1.decode(output[0], skip_special_tokens=True)
print(generated_text_1)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
 quiero verlas piensas siento mas tan solo amor cada noche voy caminar juntos lado siempre feliz ven aqui va hacerme sentir hoy vuelto puedo decirlo mireste amo vas marchar cielo nunca pierdes vida entera amar contigo vuelvas ser mejor ahora sientes ganas volver vez si miras jamas sabras dios vuelves querer verte llorare amar vivo tiempo esperar pues mia necesito cariño daria quererte olvidarnos ayer bien pronto regreses mañana estaras esperando lejos vivir aunque llegue tarde conocimos dos años querido regresarte luna primavera calor verano besos sueños tristes ilusiones traigo pensamiento extraño pensar oir miedo ir cambiando you might also like


In [8]:
generated_text_without_prompt_1 = tokenizer_1.decode(
    output[0][input_ids_1.shape[1]:],  # Quitar las tokens del prompt
    skip_special_tokens=True
)
print(generated_text_without_prompt_1)

 quiero verlas piensas siento mas tan solo amor cada noche voy caminar juntos lado siempre feliz ven aqui va hacerme sentir hoy vuelto puedo decirlo mireste amo vas marchar cielo nunca pierdes vida entera amar contigo vuelvas ser mejor ahora sientes ganas volver vez si miras jamas sabras dios vuelves querer verte llorare amar vivo tiempo esperar pues mia necesito cariño daria quererte olvidarnos ayer bien pronto regreses mañana estaras esperando lejos vivir aunque llegue tarde conocimos dos años querido regresarte luna primavera calor verano besos sueños tristes ilusiones traigo pensamiento extraño pensar oir miedo ir cambiando you might also like


In [9]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_2 = tokenizer_2(prompt, return_tensors="pt").input_ids

output = model_2.generate(
    input_ids=input_ids_2,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_2.eos_token_id,
)

generated_text_2 = tokenizer_2.decode(output[0], skip_special_tokens=True)
print(generated_text_2)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
lores hablas amor vida nunca muere solo corazon vive mejor aqui mas profundo alma amor importa si mas tiempo pasa ayer oh yeak besos negros sueños color verde gris azul rosas van muriendo buscando mirada cada beso siento soledad puedo hacer conmigo olvidarte quiero you might also like


In [10]:
generated_text_without_prompt_2 = tokenizer_2.decode(
    output[0][input_ids_2.shape[1]:],  # Quitar las tokens del prompt
    skip_special_tokens=True
)
print(generated_text_without_prompt_2)

lores hablas amor vida nunca muere solo corazon vive mejor aqui mas profundo alma amor importa si mas tiempo pasa ayer oh yeak besos negros sueños color verde gris azul rosas van muriendo buscando mirada cada beso siento soledad puedo hacer conmigo olvidarte quiero you might also like


In [11]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_3 = tokenizer_3(prompt, return_tensors="pt").input_ids

output = model_3.generate(
    input_ids=input_ids_3,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_3.eos_token_id,
)

generated_text_3 = tokenizer_3.decode(output[0], skip_special_tokens=True)
print(generated_text_3)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
- Dejo hablaran contigo hoy quiero verlos aunque nunca olvidas veces olvides ayer favor adios see luis miguel liveget tickets as low as you might also likeyo amandote tan sensual asi besarte cuerpo quiero conocerme sentirlo abrazame si busque mujer puedo entender corazon dejes ir pronto todavia quiera volver conmigo quiero saber besos beso muero vez mas amor tendre lugar cerca veo siento dentro siempre presente mientras viva voz mundo entero cambio solo silencio puedes evitar realidad pienses bien creo nadie sabre voy decir momento vas olvidar hablemos momentos diran siquiera sera seguro verdad hiciste daño quererte cada instante tambien vuelvo loco hoy fingir encuentres nuevo nombre aqui mañana mas grande ¿por ahora puedo vivir aun ser feliz ahora quieres saber baby si busques desnuda quiero conocer vida entera quiero saber cuanto mejor hacer ¿como digas sufrir perdis pasearia pasare ga

In [12]:
generated_text_without_prompt_3 = tokenizer_3.decode(
    output[0][input_ids_3.shape[1]:],  # Quitar las tokens del prompt
    skip_special_tokens=True
)
print(generated_text_without_prompt_3)

- Dejo hablaran contigo hoy quiero verlos aunque nunca olvidas veces olvides ayer favor adios see luis miguel liveget tickets as low as you might also likeyo amandote tan sensual asi besarte cuerpo quiero conocerme sentirlo abrazame si busque mujer puedo entender corazon dejes ir pronto todavia quiera volver conmigo quiero saber besos beso muero vez mas amor tendre lugar cerca veo siento dentro siempre presente mientras viva voz mundo entero cambio solo silencio puedes evitar realidad pienses bien creo nadie sabre voy decir momento vas olvidar hablemos momentos diran siquiera sera seguro verdad hiciste daño quererte cada instante tambien vuelvo loco hoy fingir encuentres nuevo nombre aqui mañana mas grande ¿por ahora puedo vivir aun ser feliz ahora quieres saber baby si busques desnuda quiero conocer vida entera quiero saber cuanto mejor hacer ¿como digas sufrir perdis pasearia pasare ganas tener sueños dia noche lado estara junto


In [13]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_4 = tokenizer_4(prompt, return_tensors="pt").input_ids

output = model_4.generate(
    input_ids=input_ids_4,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_4.eos_token_id,
)

generated_text_4 = tokenizer_4.decode(output[0], skip_special_tokens=True)
print(generated_text_4)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
En Ciudad de los sueños conmigo entiendes tan solo espero cada beso contigo amor dos caras siempre lado asi seras amigo mas querido aqui mundo extraño ahora vengo encontrar manera salir diferente sabes verdad quiero sentir cuerpo mia forma mirarme ojos darte cuenta necesito vivir dentro nuevo mejor si quieres ver luz dimelo tenerlo decir alguien escuchara hablas tantas cosas contarnos momentos daran alguna vez jamas habra historia tiempo vivido vida entera see julieta venegas liveget tickets as low as you might also like


In [ ]:
generated_text_without_prompt_4 = tokenizer_4.decode(
    output[0][input_ids_4.shape[1]:],  # Quitar las tokens del prompt
    skip_special_tokens=True
)
print(generated_text_without_prompt_4)

En Ciudad de los sueños conmigo entiendes tan solo espero cada beso contigo amor dos caras siempre lado asi seras amigo mas querido aqui mundo extraño ahora vengo encontrar manera salir diferente sabes verdad quiero sentir cuerpo mia forma mirarme ojos darte cuenta necesito vivir dentro nuevo mejor si quieres ver luz dimelo tenerlo decir alguien escuchara hablas tantas cosas contarnos momentos daran alguna vez jamas habra historia tiempo vivido vida entera see julieta venegas liveget tickets as low as you might also like


## Evaluation

In [20]:
def calculate_perplexity(model, tokenizer, text_total):
    input_ids = tokenizer(text_total, return_tensors="pt").input_ids

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = torch.exp(loss)
    
    return perplexity.item()


In [21]:
def distinct_n(text, n):
    tokens = text.split()
    ngrams = zip(*[tokens[i:] for i in range(n)])
    unique_ngrams = set(ngrams)
    return len(unique_ngrams) / len(tokens) if tokens else 0

In [22]:
def lexical_diversity(text):
    tokens = word_tokenize(text)
    unique_tokens = set(tokens)
    return len(unique_tokens) / len(tokens) if tokens else 0

In [23]:
# Load a pre-trained embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def local_coherence(text):
    sentences = text.split('.')
    coherence_scores = []
    for i in range(len(sentences) - 1):
        emb1 = embedding_model.encode(sentences[i], convert_to_tensor=True)
        emb2 = embedding_model.encode(sentences[i + 1], convert_to_tensor=True)
        similarity = util.cos_sim(emb1, emb2).item()
        coherence_scores.append(similarity)
    return sum(coherence_scores) / len(coherence_scores) if coherence_scores else 0

In [44]:
def evaluate_text(model, tokenizer, text_total, text_generated):
    return {
        "Perplexity": calculate_perplexity(model,tokenizer, text_total),
        "Distinct-2": distinct_n(text_generated, 2),
        "Lexical Diversity": lexical_diversity(text_generated),
        "Local Coherence": local_coherence(text_total),
    }

In [45]:
metrics_1 = evaluate_text(model_1, tokenizer_1, generated_text_1, generated_text_without_prompt_1)
metrics_1

{'Perplexity': 71.47877502441406,
 'Distinct-2': 0.9900990099009901,
 'Lexical Diversity': 0.9900990099009901,
 'Local Coherence': 0.3658442199230194}

In [46]:
metrics_2 = evaluate_text(model_2, tokenizer_2, generated_text_2, generated_text_without_prompt_2)
metrics_2

{'Perplexity': 87.86155700683594,
 'Distinct-2': 0.9791666666666666,
 'Lexical Diversity': 0.9583333333333334,
 'Local Coherence': 0.4929533004760742}

In [47]:
metrics_3 = evaluate_text(model_3, tokenizer_3, generated_text_3, generated_text_without_prompt_3)
metrics_3

{'Perplexity': 42.923221588134766,
 'Distinct-2': 0.9864864864864865,
 'Lexical Diversity': 0.918918918918919,
 'Local Coherence': 0.3888491690158844}

In [48]:
metrics_4 = evaluate_text(model_4, tokenizer_4, generated_text_4, generated_text_without_prompt_4)
metrics_4

{'Perplexity': 36.7916145324707,
 'Distinct-2': 0.9880952380952381,
 'Lexical Diversity': 0.9880952380952381,
 'Local Coherence': 0.44628340005874634}